In [1]:
import os
import rasterio
import tifftools
import rasterio.mask
import numpy as np
from contextlib import contextmanager  
from rasterio import Affine
from rasterio.enums import Resampling
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import fiona
import geopandas as gpd
from rasterio.features import shapes
from osgeo import gdal
from osgeo import ogr
from osgeo import gdalconst


def mkdir(dir):
    
    if not os.path.exists(dir):
        os.mkdir(dir)

path_run = os.getcwd() + '/'
path_data = path_run + 'data/'

path_data_processed = path_run + 'data_processed/'
path_data_processed_creator = mkdir(path_data_processed)
path_population_processed = path_data_processed + 'population_processed/'
path_population_processed_creator = mkdir(path_population_processed)
path_GeoClaw_processed_shp = path_data_processed + 'GeoClaw_processed_shp/'
path_GeoClaw_processed_shp_creator = mkdir(path_GeoClaw_processed_shp)
path_MERIT_DEM_processed = path_data_processed + 'MERIT_DEM_processed/'
path_MERIT_DEM_processed_creator = mkdir(path_MERIT_DEM_processed)
path_RICorDE_processed = path_data_processed + 'RICorDE_processed/'
path_RICorDE_processed_creator = mkdir(path_RICorDE_processed)
path_RICorDE_processed_layer = path_RICorDE_processed + 'RICorDE_layer/'
path_RICorDE_processed_layer_creator = mkdir(path_RICorDE_processed_layer)
path_RICorDE_processed_layer_tif_processed = path_RICorDE_processed_layer + 'tif/'
path_RICorDE_processed_layer_tif_creator = mkdir(path_RICorDE_processed_layer_tif_processed)
path_RICorDE_processed_layer_shp_processed = path_RICorDE_processed_layer + 'shp/'
path_RICorDE_processed_layer_shp_creator = mkdir(path_RICorDE_processed_layer_shp_processed)

crs = {'init' :'epsg:4326'}     

# Resample Satellite 


In [2]:
# Change NaN to 0s

raster_layer = rasterio.open(path_data + 'RICorDE/' + "idai_r1_0331_depths.tif").read()
raster_profile = rasterio.open(path_data + 'RICorDE/' + "idai_r1_0331_depths.tif").profile

raster_layer[raster_layer<0] = 0

with rasterio.open((path_RICorDE_processed + "idai_r1_0331_depths_0.tif"),'w', **raster_profile) as dst:
    dst.write(raster_layer)

In [3]:
# Resample Satellite 

# define 'dat' and 'pixel_transformer'

raster_layer_resampled = path_RICorDE_processed + "idai_r1_0331_depths_0_resampled_9as.tif"
pixel_transformer_tif = 1/3

# Define Resampling Function #

@contextmanager
def resample_raster(raster, scale = pixel_transformer_tif):
    t = raster.transform

    # rescale the metadata

    transform = Affine(t.a / scale, t.b, t.c, t.d, t.e / scale, t.f)
    height = raster.height * scale
    width = raster.width * scale

    profile = src.profile
    profile.update(transform=transform, driver='GTiff', height=height, width=width, nodata=np.nan)
    print(profile)
      
    data = raster.read( # Note changed order of indexes, arrays are band, row, col order not row, col, band
            out_shape=(int(raster.count), int(height), int(width)),
            resampling=Resampling.average)
    
    data = np.round(data,1)

    with rasterio.open((raster_layer_resampled),'w', **profile) as dst:
        dst.write(data)
        yield data

# Apply Function by defining scaling factor        

with rasterio.open(path_RICorDE_processed + "idai_r1_0331_depths_0.tif") as src:
    with resample_raster(src, pixel_transformer_tif) as resampled:
        print('Orig dims: {}, New dims: {}'.format(src.shape, resampled.shape))

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 2611.333333333333, 'height': 2746.6666666666665, 'count': 1, 'crs': CRS.from_epsg(32737), 'transform': Affine(270.0, 0.0, -308320.1684614606,
       0.0, -270.0, 8341431.159002916), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
Orig dims: (8240, 7834), New dims: (1, 2746, 2611)


# Reproject satellite imagery

In [4]:
dst_crs = 'EPSG:4326'

with rasterio.open(path_RICorDE_processed + "idai_r1_0331_depths_0_resampled_9as.tif") as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(path_RICorDE_processed + "idai_r1_0331_depths_0_resampled_9as_reproj.tif", 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

# Create outline GeoClaw 

In [5]:
GeoClaw_shape = path_GeoClaw_processed_shp + f"GeoClaw_outline.shp"

mask = None

with rasterio.Env():
    with rasterio.open(path_data + '/GeoClaw/shp_template/cfwindzos065_no_dif.tif') as src:
        image = src.read()
        image[image<1000] = 0

    results = (
    {'properties': {'raster_val': v}, 'geometry': s}
    for i, (s, v)
    in enumerate(
    shapes(image, mask=mask, transform=src.transform)))

    with fiona.open(
    GeoClaw_shape, 'w',
    driver='Shapefile',
    crs=src.crs,
    schema={'properties': [('raster_val', 'int')],
    'geometry': 'Polygon'}) as dst:
        dst.writerecords(results)

# Create outline GeoClaw - to adjust pixel dimesions

In [6]:
# cropping alternative - implemented above code

mask = None  

path_raster_layer_resampled = path_data + 'GeoClaw/shp_template/cfwindzos065_no_dif.tif'
GeoClaw_shape = path_data + f"GeoClaw/shp_template/GeoClaw_outline.shp"

with fiona.open(GeoClaw_shape, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(path_raster_layer_resampled) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
                  "height": out_image.shape[1],
                  "width": out_image.shape[2],
                  "transform": out_transform})

out_image[out_image==0] = np.nan
out_image_RICorDE = out_image.copy()
path_raster_layer_resampled_cropped = path_RICorDE_processed + '/cfwindzos065_no_dif_cropped_2.tif'

with rasterio.open(path_raster_layer_resampled_cropped, "w", **out_meta) as dest:
        dest.write(out_image) 

# Extract depth values and burn into GeoClaw raster

In [ ]:
# Create shp out of satellite tif

for cm in range(10,1010,10):

    cm = cm/100
    print(cm)

#     RICorDE_shape_pre = path_RICorDE_processed_layer_shp_processed + f"idai_r1_0331_depths_0_resampled_9as_reproj_{cm}_pre.shp"
    RICorDE_shape = path_RICorDE_processed_layer_shp_processed + f"idai_r1_0331_depths_0_resampled_9as_reproj_{cm}.shp"

    mask = None

    from rasterio.features import shapes

    with rasterio.Env():
        with rasterio.open(path_RICorDE_processed + "idai_r1_0331_depths_0_resampled_9as_reproj.tif") as src:
            image = src.read()

        
            image[np.isnan(image)] = 0
            image[image<cm] = 0
            image[image>=cm] = 1

        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v)
        in enumerate(
        shapes(image, mask=mask, transform=src.transform)))

        with fiona.open(
        RICorDE_shape, 'w',
        driver='Shapefile',
        crs=src.crs,
        schema={'properties': [('raster_val', 'int')],
        'geometry': 'Polygon'}) as dst:
            dst.writerecords(results)

    satellite_shape_dissolved = gpd.read_file(RICorDE_shape)
    satellite_shape_dissolved = satellite_shape_dissolved[satellite_shape_dissolved['raster_val'] == 1]
    satellite_shape_dissolved.to_file(RICorDE_shape)


    rst_fn = path_RICorDE_processed + '/cfwindzos065_no_dif_cropped_2.tif'
    shp_fn = RICorDE_shape

    #This raster is the model for our output (CRS, extent)
    ndsm = rst_fn
    #This shapefile contains the features we want to burn
    shp = shp_fn

    data = gdal.Open(ndsm, gdalconst.GA_ReadOnly)

    geo_transform = data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * data.RasterXSize
    y_min = y_max + geo_transform[5] * data.RasterYSize
    x_res = data.RasterXSize
    y_res = data.RasterYSize


    vec = ogr.Open(shp)
    lyr = vec.GetLayer(0)
    pixel_width = geo_transform[1]

    output = path_RICorDE_processed_layer_tif_processed + f"idai_r1_0331_depths_resampled_9as_reproj_{cm}.tif"
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_UInt32)
    target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
    band = target_ds.GetRasterBand(1)
    NoData_value = 0
    band.SetNoDataValue(NoData_value)
    band.FlushCache()


    driver = ogr.GetDriverByName('ESRI Shapefile')
    for feat in lyr:
        
        burn_value = int(feat.GetField("raster_val"))
        datasource = driver.CreateDataSource(path_data_processed + '/RICorDE_processed/RICorDE_layer/temp.shp')
        layer = datasource.CreateLayer('temp', lyr.GetSpatialRef(), geom_type=ogr.wkbPolygon)
        layer.CreateFeature(feat)
        gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[burn_value], options=["ALL_TOUCHED=FALSE"])
        datasource.Destroy()

    #This makes raster to write to disk
    
    target_ds = None    

# Summarize all depth layer to one final satellite depth layer

In [ ]:
sat_final = rasterio.open(path_RICorDE_processed_layer_tif_processed + f"idai_r1_0331_depths_resampled_9as_reproj_0.1.tif").read()
sat_final[:] = 0
sat_final_profile = rasterio.open(path_RICorDE_processed_layer_tif_processed + f"idai_r1_0331_depths_resampled_9as_reproj_0.1.tif").profile
sat_final_profile['dtype'] = 'float32'
sat_final_profile['crs'] = crs

for cm in range(10,1010,10):
    
    cm = cm/100
    print(cm)
    
    sat_temp = rasterio.open(path_RICorDE_processed_layer_tif_processed + f"idai_r1_0331_depths_resampled_9as_reproj_{cm}.tif").read()
    sat_final = sat_final+sat_temp   
    
sat_final = np.float32(sat_final/10)
sat_final[sat_final==0] = np.nan 
profile_sat = out_meta.copy()
profile_sat
with rasterio.open(path_RICorDE_processed + f"idai_r1_0331_depths_0_resampled_9as_reproj_final.tif", "w", **sat_final_profile) as dest:
        dest.write(sat_final) 